# Pipeline

This is meant to be a more pipelined version of the `bullet_journal_llm` notebook experiment. This will
do bulk extraction of notes from the notebooks, with different prompts based on the notebook.

In [ ]:
from pathlib import Path
import logging
import sys
import yaml

import google.generativeai as genai
import os

_LOGGER = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

SECRETS = Path("../secrets.yaml")
MODEL_ID = 'gemini-1.5-flash'

secrets = yaml.safe_load(SECRETS.read_text())
genai.configure(api_key=secrets["gemini_api_key"])

_LOGGER.info("Initializing client library")
model = genai.GenerativeModel(MODEL_ID)

In [ ]:
NOTES_DIR = Path("../notes")
PAGES_DIR = Path("../pages")


In [2]:
PROFILE_PROMPT = """Allen Porter, software engineer from Los Angeles, CA. Uses the notebook to track work, hobbies, and family obligations."""

TASK_LEGEND = """
Here is a legend for the different types of bullets used in the Bullet Journal method:
  o event
  • task
* • critical task
  X completed task
  - note
  < moved to future log
  > moved to another daily, weekly, or monthly note

Note that * can be added to any task to indicate it is a critical task (e.g. a critical task can be moved or completed)
"""

GENERIC_PROMPT = """You are analyzing PNG images of handwritten text from an
e-ink notebook SuperNote A5X. The notes are written in English and are in a
bullet journal format. You can see that the text is not perfect and will need
some cleaning up

The Bullet Journal method is a system that combines elements of mindfulness,
productivity, and self-discovery. It empowers the user to become the author of their
own life, allowing them to track the past, organize the present, and plan for the
future. A Bullet Journal method may be described as a productivity system or an
organization system, but at its core, the method is a tool for changing the way
we approach our day-to-day tasks and long term goals. The Bullet Journal method
is centered on one key idea: intentionality. Why do we do what we do? What makes
these goals meaningful to us? What tasks are the most relevant to  us at any
given point in time? 

Rapid logging is the language of the bullet journal method and it functions
through the use of  Bullets to indicate a task's status. A task starts with a simple
dot "•" to represent a task. If a task is completed, mark it with an "X". If it's
migrated to a future date, use a right arrow (>) to indicate that. And additional
bullet styles can be used depending on what makes sense to the author.

Tasks within the Bullet Journal method can then fall within any of the logs used
depending on where they fall in the author's timeline. Typically, journals contain
a Daily Log, Weekly Log, and Monthly Log.

Daily Log
The daily log is used to record tasks, events, and notes on a day-to-day
basis. A daily log is simply the current date as the header and then a list of
log entires. They can be short, bulleted entries with symbols to represent
different types of content (tasks, events, notes).

When new tasks arise throughout the day, they are add them to the daily log. If
a task is completed, it is marked with an "X". If a task is not completed and the
author still wants to work on it, they can migrate it to a future date by using
a right arrow (>) to indicate that. This way, the author maintains a concise and up-to-date
record of their daily activities.

Weekly Log
The weekly log provides an overview of the week, allowing the author to plan and
organize tasks, events, and commitments in a broader context. A weekly log 
is started by writing the dates of the week at the top of the page. Below are
typically a list of tasks and notes for the week, using bulleted entries. At
the end of the week, the author can migrate unfinished tasks to the next week
as needed.

The weekly log serves as a snapshot of the upcoming week, helping prioritize
tasks, allocate time, and have a holistic view of the author's commitments.

A weekly log may also be accompanied by a weekly review, where the author
reflects on the previous week's accomplishments, challenges, and goals for the upcoming
week.

Monthly Log
The monthly log offers a broader perspective, enabling the author to plan and
track their activities, goals, and events throughout the month. To create a monthly
log, designate a double-page spread in their notebook for each month.

A monthly spread has the name of the month at the top of the list. The left-hand
page has a list of dates (1-30/31) down the side. This space is used to mark important
events, deadlines, or appointments for each day. The right-hand page is used as
a monthly last list, where the author can list out tasks, goals, or projects they
want to focus on during the month.

The monthly log serves as a reference point, allowing the author to see the big
picture and plan ahead. As the month progresses, they can refer to this log, migrate
tasks to specific days in their daily log or add new tasks as they arise.

{TASK_LEGEND}

Author profile: {PROFILE_PROMPT}
"""

In [4]:

DAILY_PROMPT = GENERIC_PROMPT + """
{daily_prompt}
{daily_content}

A notebook page should be converted to json data with the following structure for the above example page:
```json
{daily_json}
```

Your job is to convert the image to text and then to json format. Only answer with the json data,
and no other commentary, as the answer will be programtically parsed.
"""


In [ ]:

WEEKLY_PROMPT = GENERIC_PROMPT + """
{weekly_prompt}
{weekly_content}

A notebook page should be converted to json data with the following structure for the above example page:
```json
{weekly_json}
```

Your job is to convert the image to text and then to json format. Only answer with the json data,
and no other commentary, as the answer will be programtically parsed.
"""

response = model.generate_content([IMG_PROMPT, *weekly_pngs])